In [1]:
from io import BytesIO
import json
import PIL.Image
import numpy as np
import requests

from velour.client import Client, ClientException
from velour.client import Dataset as VelourDataset, Model as VelourModel
from velour.schemas import Image
from velour.viz import combined_segmentation_mask
from velour.integrations.coco import upload_coco_panoptic
from velour.integrations.yolo import parse_yolo_object_detection, parse_yolo_image_segmentation
from velour.metrics import Task

from ultralytics import YOLO

/home/czaloom/velour/.env-velour/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from chariot.datasets import Dataset as ChariotDataset

In [3]:
client = Client("http://localhost:8000")

Succesfully connected to http://localhost:8000/.


# Dataset ingestion

We assume the COCO panoptic evaluations from
http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip have been downloaded and unzipped (including unzipping the file `panoptic_val2017.zip`) to `./coco`

In [4]:
with open("./coco/annotations/panoptic_val2017.json") as f:
    annotations = json.load(f)
    
annotations["annotations"] = annotations["annotations"][:10]

In [5]:
# helper method to download underlying images from COCO
image_id_to_coco_url = {str(img_dict["id"]): img_dict["coco_url"] for img_dict in annotations["images"]}
def download_image(img: Image) -> PIL.Image:
    url = image_id_to_coco_url[img.uid]
    img_data = BytesIO(requests.get(url).content)
    return PIL.Image.open(img_data)

In [6]:
# creates a dataset object in velour
try:
    dataset = VelourDataset.create(client, "coco2017-panoptic")
    upload_coco_panoptic(
        dataset,
        annotations=annotations,
        masks_path="./coco/annotations/panoptic_val2017/"
    )
except ClientException:
    dataset = VelourDataset.get(client, "coco2017-panoptic")

In [7]:
dataset.get_images()

[]

In [8]:
download_image(dataset.get_images()[0])

IndexError: list index out of range

In [ ]:
instance_segs = dataset.get_groundtruth_instance_segmentations("139")
semantic_segs = dataset.get_groundtruth_semantic_segmentations("139")

In [ ]:
instance_mask, instance_legend = combined_segmentation_mask(instance_segs, label_key="name")

In [ ]:
instance_mask

In [ ]:
for k, v in instance_legend.items():
    print(k)
    display(v)

In [ ]:
semantic_mask, semantic_legend = combined_segmentation_mask(semantic_segs, label_key="name")

In [ ]:
semantic_mask

In [ ]:
for k, v in semantic_legend.items():
    print(k)
    display(v)

# Model inference

In [ ]:
from chariot.client import connect
from chariot.models import Model
from tqdm.auto import tqdm

from velour.integrations.chariot import create_model_from_chariot, parse_chariot_object_detections

In [ ]:
connect("https://production.chariot.striveworks.us/")

In [ ]:
dataset = client.get_dataset("coco2017-panoptic")

In [ ]:
model_name = "fasterrcnnresnet-50fpn"
chariot_model = Model(name=model_name, project_name="Global")

try:
    velour_chariot_model = create_model_from_chariot(client, chariot_model)
except ClientException:
    velour_chariot_model = client.get_model('chariot-'+model_name+'-v'+str(chariot_model.version))


try:
    for image_metadata in tqdm(dataset.get_images()):
        image = download_image(image_metadata)    
        
        # Chariot Inference
        detections = chariot_model.detect(image)
        velour_dets = parse_chariot_object_detections(detections, image_metadata, label_key="name") 

        velour_chariot_model.add_predictions(dataset=dataset, predictions=velour_dets, show_progress_bar=False)
except ClientException:
    print("Velour Chariot model already finalized.")
    

In [ ]:
model_name = "yolov8n-seg"
yolo_model = YOLO(f"{model_name}.pt")

try:
    velour_yolo_model = client.create_image_model(model_name + "")
except ClientException:
    velour_yolo_model = client.get_model(model_name + '')

try:
    for image_metadata in tqdm(dataset.get_images()):
        
        # YOLO Inference (before)
        image = download_image(image_metadata)
        results = yolo_model(image, verbose=False)

        velour_dets = parse_yolo_object_detection(results[0], uid=image_metadata.uid, label_key='name')
        velour_yolo_model.add_predictions(dataset=dataset, predictions=velour_dets, show_progress_bar=False)

except ClientException:
    print("Velour YOLO model already finalized.")

In [ ]:
print(velour_dets[0].scored_labels)

In [ ]:
datums = []
def bulk_inference():
    pass
def parse_bulk_inference():
    pass
def convert_to_bulk_inference_format():
    pass
model = None
class Datum:
    pass
yolo: callable = None
parse_coco_panoptic = bulk_inference
coco_to_velour = None
yolo_to_velour = None

"""

"""

with open("./coco/annotations/panoptic_val2017.json") as f:
    datums = json.load(f)

## Dataset Creation
dataset = client.create_dataset("coco2017-panoptic")        # NOTE: Dataset/Model will determine type based on the data uploaded to it. 
dataset.metadata["href"] = "some url"
dataset.metadata["description"] = "the coco dataset"
dataset.metadata["project"] = "some project id"
dataset.metadata["can be anything"] = 123

## Ground Truth
for image in annotations:

    # Create Datum
    datum = dataset.create_datum(image.uid)
    datum.metadata["type"] = "image"
    datum.metadata["lens_zoom"] = 0.4

    # GroundTruths (Custom)
    gts = [                                                 # This for-loop is what a parsing integration abstracts
        coco_to_velour(detection)                           # NOTE: Still need some form of object conversion
        for detection in image
    ]
    dataset.add_groundtruths(datum, gts)

dataset.finalize()

## Model Creation
model = client.create_model("yolov8n-seg")
model.metadata["href"] = "some url"
model.metadata["description"] = "yolo"

## Inference
for datum in tqdm(dataset.get_images()):

        # 3rd-Party Inference
        image = download_image(datum)
        results = yolo(image, verbose=False)
 
        # Predictions (Custom)
        preds = [                                           # This for-loop is what a parsing integration abstracts
            yolo_to_velour(detection)                       # NOTE: Still need some form of object conversion
            for detection in results                
        ]
        model.add_predictions(datum, preds)

model.finalize()

# Evaluation

for provenance, the dataset and inferences on the dataset must be finalized before evaluation can be done

In [ ]:
dataset.finalize()
velour_chariot_model.finalize_inferences(dataset)
velour_yolo_model.finalize_inferences(dataset)

In [ ]:
import time

def evaluate_ap(dataset, model, max_area=None):
    eval_job = model.evaluate_ap(
        dataset=dataset,
        model_pred_task_type=Task.BBOX_OBJECT_DETECTION,
        dataset_gt_task_type=Task.INSTANCE_SEGMENTATION,
        label_key="name",
        max_area=max_area,
    )

    while eval_job.status() == 'Processing':
        time.sleep(1)
    
    return eval_job

In [ ]:
job1 = evaluate_ap(dataset, velour_chariot_model)
job2 = evaluate_ap(dataset, velour_chariot_model, max_area=30*300)

In [ ]:
job3 = evaluate_ap(dataset, velour_yolo_model)
job4 = evaluate_ap(dataset, velour_yolo_model, max_area=30*300)

In [ ]:
print(job1.metrics()[:2])

In [ ]:
print(job2.metrics()[:2])

In [ ]:
# print(job3.metrics()[:2])
for m in job3.metrics():
    print(m)

In [ ]:
print(job4.metrics()[:2])

# Metadata

In [ ]:
from velour.client import Client
from velour.data_types import Label
client = Client("http://localhost:8000")

velour_coco = client.get_dataset("coco2017-panoptic")

model_name = "yolov8n-seg"
velour_yolo = client.get_model(model_name + '')

model_name = "fasterrcnnresnet-50fpn"
velour_chariot = client.get_model('chariot-'+model_name+'-v'+str('1'))

In [ ]:
print(velour_coco.name)
print(velour_coco.description)
print(velour_coco.href)
info = velour_coco.get_info()
print(info.number_of_classifications)
print(info.number_of_bounding_boxes)
print(info.number_of_bounding_polygons)
print(info.number_of_segmentations)
print(info.annotation_type)
print(info.associated_models)

In [ ]:
print(velour_yolo.name)
print(velour_yolo.description)
print(velour_yolo.href)
info = velour_yolo.get_info()
print(info.number_of_classifications)
print(info.number_of_bounding_boxes)
print(info.number_of_bounding_polygons)
print(info.number_of_segmentations)
print(info.annotation_type)
print(info.associated_datasets)

In [ ]:
print(velour_chariot.name)
print(velour_chariot.description)
print(velour_chariot.href)
info = velour_chariot.get_info()
print(info.number_of_classifications)
print(info.number_of_bounding_boxes)
print(info.number_of_bounding_polygons)
print(info.number_of_segmentations)
print(info.annotation_type)
print(info.associated_datasets)

In [ ]:
distribution = velour_coco.get_label_distribution()

label_person = Label(key='name', value='person')
label = Label(key='supercategory', value='furniture')

count = distribution[label_person]

print(count)

In [ ]:
distribution = velour_yolo.get_label_distribution()

label = Label(key='name', value='tv')

info = distribution[label]

print(info['count'])
print(info['scores'])

In [ ]:
labels = velour_coco.get_labels()

for label in labels:
    print(label)

In [ ]:
labels = velour_yolo.get_labels()

for label in labels:
    print(label)